In [2]:
import tensorflow as tf
import keras
from keras.layers import Dense, Input, Conv2D, BatchNormalization, MaxPool2D, Flatten
from keras.models import Model
from keras.utils import to_categorical, multi_gpu_model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
train = pd.read_csv('train.csv')
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
labels = train.label.to_numpy().reshape((42000,1))
images = train.drop('label', axis=1).to_numpy().reshape((42000,28,28,1))
images = images / 255
#plt.imshow(images[0])
labelsEncoding = to_categorical(labels)

In [4]:
def convModel():
    inputs = Input(shape=(28,28,1))
    
    X = Conv2D(32, 3, padding='same')(inputs)
    X = MaxPool2D(pool_size=(2,2))(X)
    
    X = Conv2D(64, 3, padding='same')(X)
    X = MaxPool2D(pool_size=(2,2))(X)
    
    X = Flatten()(X)
    
    X = Dense(32, kernel_regularizer='l2', activation='relu')(X)
    X = Dense(32, kernel_regularizer='l2', activation='relu')(X)
    
    outputs = Dense(10, kernel_regularizer='l2', activation='softmax')(X)
    
    model = Model(inputs = inputs, outputs = outputs)
    return model

In [5]:
model = convModel()
model = multi_gpu_model(model, gpus=2)
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
model.fit(images, labelsEncoding, batch_size=128, epochs=10)
model.save_weights('conv_nobatchnorm.h5')

Epoch 1/10
42000/42000 [==============================] - 3s 78us/step - loss: 0.8816 - acc: 0.8762
Epoch 2/10
42000/42000 [==============================] - 1s 34us/step - loss: 0.4295 - acc: 0.9612
Epoch 3/10
42000/42000 [==============================] - 1s 34us/step - loss: 0.3322 - acc: 0.9704
Epoch 4/10
42000/42000 [==============================] - 1s 34us/step - loss: 0.2798 - acc: 0.9740
Epoch 5/10
42000/42000 [==============================] - 1s 34us/step - loss: 0.2502 - acc: 0.9743
Epoch 6/10
42000/42000 [==============================] - 1s 34us/step - loss: 0.2242 - acc: 0.9757
Epoch 7/10
42000/42000 [==============================] - 1s 34us/step - loss: 0.2083 - acc: 0.9764
Epoch 8/10
42000/42000 [==============================] - 1s 34us/step - loss: 0.1943 - acc: 0.9771
Epoch 9/10
42000/42000 [==============================] - 1s 34us/step - loss: 0.1840 - acc: 0.9774
Epoch 10/10
42000/42000 [==============================] - 1s 35us/step - loss: 0.1764 - acc: 0.9782

In [6]:
test = pd.read_csv('test.csv')
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
testimages = test.to_numpy().reshape((28000,28,28,1))
testimages = testimages/255

In [8]:
predictions = model.predict(testimages, batch_size=128, verbose=1)

28000/28000 [==============================] - 0s 16us/step


In [17]:
predictionLabels = pd.DataFrame({'Label':predictions.argmax(axis=-1).reshape((28000,1))[:,0]})
ImageId = pd.DataFrame({'ImageId': [i+1 for i in range(28000)]})
submission = pd.concat([ImageId, predictionLabels], axis = 1)
submission.to_csv('submission_conv_nobatchnorm.csv', index=False)

In [34]:
def convModelBatchNorm():
    inputs = Input(shape=(28,28,1))
    
    X = Conv2D(32, 3, padding='same', activation='relu')(inputs)
    X = BatchNormalization()(X)
    X = MaxPool2D(pool_size=(2,2))(X)
    
    X = Conv2D(64, 3, padding='same', activation='relu')(X)
    X = BatchNormalization()(X)
    X = MaxPool2D(pool_size=(2,2))(X)
    
    X = Flatten()(X)
    
    X = Dense(128, kernel_regularizer='l2', activation='relu')(X)
    X = Dense(128, kernel_regularizer='l2', activation='relu')(X)
    
    outputs = Dense(10, kernel_regularizer='l2', activation='softmax')(X)
    
    model = Model(inputs = inputs, outputs = outputs)
    return model

In [35]:
model = convModelBatchNorm()
model = multi_gpu_model(model, gpus=2)
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
model.fit(images, labelsEncoding, batch_size=128, epochs=10)
model.save_weights('conv_batchnorm_relu.h5')

Epoch 1/10
42000/42000 [==============================] - 3s 64us/step - loss: 1.5157 - acc: 0.9540
Epoch 2/10
42000/42000 [==============================] - 2s 44us/step - loss: 0.3326 - acc: 0.9837
Epoch 3/10
42000/42000 [==============================] - 2s 44us/step - loss: 0.2037 - acc: 0.9863
Epoch 4/10
42000/42000 [==============================] - 2s 44us/step - loss: 0.1687 - acc: 0.9884
Epoch 5/10
42000/42000 [==============================] - 2s 44us/step - loss: 0.1550 - acc: 0.9892
Epoch 6/10
42000/42000 [==============================] - 2s 44us/step - loss: 0.1462 - acc: 0.9896
Epoch 7/10
42000/42000 [==============================] - 2s 44us/step - loss: 0.1361 - acc: 0.9904
Epoch 8/10
42000/42000 [==============================] - 2s 44us/step - loss: 0.1336 - acc: 0.9906
Epoch 9/10
42000/42000 [==============================] - 2s 44us/step - loss: 0.1255 - acc: 0.9916
Epoch 10/10
42000/42000 [==============================] - 2s 44us/step - loss: 0.1214 - acc: 0.9926

In [36]:
predictions_batchnorm = model.predict(testimages, batch_size=128, verbose=1)

28000/28000 [==============================] - 1s 30us/step


In [37]:
predictionLabels_batchnorm = pd.DataFrame({'Label':predictions_batchnorm.argmax(axis=-1).reshape((28000,1))[:,0]})
ImageId_batchnorm = pd.DataFrame({'ImageId': [i+1 for i in range(28000)]})
submission_batchnorm = pd.concat([ImageId_batchnorm, predictionLabels_batchnorm], axis = 1)
submission.to_csv('submission_conv_batchnorm_relu2.csv', index=False)

In [38]:
predictionLabels.equals(predictionLabels_batchnorm)

False